> Since the OpenBMI data has 54 subjects and the sampling rate is 1000Hz, using `moabb` to read all subjects into memory at one time and then perform corresponding preprocessing on the data set may cause excessive memory usage. However, `dpeeg` has not yet implemented a method to save memory by reading one subject at a time during training (a data set download similar to `moabb` may be added in the future to customize the data set format of `dpeeg` to achieve more refinement Experimental method), so this article adopts a compromise method to preprocess all the data and then read it into the memory at once.

# Session-Independent

In [ ]:
import mne
import numpy as np
from moabb.datasets import Lee2019_MI

OpenBMI = Lee2019_MI(test_run=True)
picks = [
    'FC5', 'FC3', 'FC1', 'FC2', 'FC4', 'FC6', 
    'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 
    'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6'
]
event_id = {'left_hand': 1, 'right_hand': 0}
dataset = {}

for sub in range(1, 55):
    print('-'*15, sub, '-'*15)
    dataset[sub] = {}
    sub_data = OpenBMI.get_data([sub])

    for sess, runs in sub_data[sub].items():
        data_list, label_list = [], []
        for raw in runs.values():
            raw.resample(250)
            events = mne.find_events(raw)
            events[:, -1] -= 1
            epochs = mne.Epochs(raw, events, event_id, 0, 4, None, picks, preload=True)
            epochs_data = epochs.get_data()[:, :, :-1] * 1e6
            data_list.append(epochs_data)
            label_list.append(events[:, -1])

        data = np.concatenate(data_list)
        label = np.concatenate(label_list)
        if sess == 'session_1':
            dataset[sub]['train'] = [data, label]
        elif sess == 'session_2':
            dataset[sub]['test'] = [data, label]
        else:
            raise KeyError(f'Unable to parse {sess}.')

from dpeeg.data import transforms

trans = transforms.Save('openbmi_session-independent')
trans(dataset)

# Session-Dependent

In [ ]:
import mne
import numpy as np
from moabb.datasets import Lee2019_MI

OpenBMI = Lee2019_MI(test_run=True)
picks = [
    'FC5', 'FC3', 'FC1', 'FC2', 'FC4', 'FC6', 
    'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 
    'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6'
]
event_id = {'left_hand': 1, 'right_hand': 0}
dataset = {}

for sub in range(1, 55):
    print('-'*15, sub, '-'*15)
    dataset[sub] = {}
    sub_data = OpenBMI.get_data([sub])

    for sess, runs in sub_data[sub].items():
        if sess == 'session_2':
            continue

        data_list, label_list = [], []
        for raw in runs.values():
            raw.resample(250)
            events = mne.find_events(raw)
            events[:, -1] -= 1
            epochs = mne.Epochs(raw, events, event_id, 0, 4, None, picks, preload=True)
            epochs_data = epochs.get_data()[:, :, :-1] * 1e6
            data_list.append(epochs_data)
            label_list.append(events[:, -1])

        data = np.concatenate(data_list)
        label = np.concatenate(label_list)
        if sess == 'session_1':
            dataset[sub] = [data, label]
        else:
            raise KeyError(f'Unable to parse {sess}.')

from dpeeg.data import transforms

trans = transforms.ComposeTransforms(
    transforms.SplitTrainTest(),
    transforms.Save('openbmi_session-dependent')
)
trans(dataset)

In [ ]:
from dpeeg.data.functions import load

dataset = load('./openbmi_session-dependent/')

In [ ]:
dataset[1]['train'][0].shape